# Основные этапы обработки Landsat

1. [Предобработка данных](005_Preprocessing.ipynb).
2. Поиск рубок.

# Названия растров на разных этапах обработки

Поскольку обработка состоит из большого числа этапов, то удобно все результаты того или иного преобразования сцены называть по одной схеме:

Номер |Этап | Пример названия|Документ
-----|-----------------------|-
1 | Импорт | от этого этапа остаются только LC81120282015189LGN00_BQA, остальные удалены.|[000_Initialization.ipynb](000_Initialization.ipynb)
2 | Расчет ToAR для импортированных растров из этапа 1. Все значения, выпадающие из диапазона [0, 1], считаются ошибочными и приводятся к соответствующей границе (нулю или единице). | toar_LC81120272015045LGN00_B9|[000_Initialization.ipynb](000_Initialization.ipynb)
3 | Топографическая коррекция для растров из этапа 2 | topo.toar_LC81120272015045LGN00_B6|[006_TopoCorrection.ipynb](006_TopoCorrection.ipynb)
4 | Применена маска облачности к растрам из этапа 1 (без топокоррекции). Все, что под облаками - NULL | clean.LC81130272016055LGN00_B3|
5 | Применена маска облачности к растрам из этапа 3 (c топокоррекцией). Все, что под облаками - NULL | clean.topo.toar_LC81120272015045LGN00_B6|


# Вспомогательные данные

## Растровые

Номер| Данные| Описание |Location/Mapset | Название карты | Документ
-|----|--|-----------------------|--|--
1|SRTM | Импортированы два растра высот 'srtm_63_03', 'srtm_64_03' и склеены в одно покрытие|FOREST/srtm | srtm| [006_TopoCorrection.ipynb](006_TopoCorrection.ipynb)
2| Покрытие лесами | Слой treecover  https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.2.html (покрытие лесов) за 2000г. | FOREST/treecover| treecover| [000_Initialization.ipynb](000_Initialization.ipynb)
3a| Маска лесов | Переклассифицированный слой из п.2: все, что меньше 30% сброшено в 0, остальное в 1. | FOREST/treecover| forest.mask.30| [000_Initialization.ipynb](000_Initialization.ipynb)
3b| Маска лесов | Переклассифицированный слой из п.2: все, что меньше 60% сброшено в 0, остальное в 1. | FOREST/treecover| forest.mask.60| [000_Initialization.ipynb](000_Initialization.ipynb)
4a| Псевдоинвариантные пиксели сцены| Берется растр clean.topo.toar_xxx_Bx и второй растр того же path/row но за другую дату. Расчитываются PIF и сохраняются в растр. Затем процедура повторяется для остальных растров того же path/row. В итоге получаем множество растров потенциальных PIF для выбранной сцены| landsat| Пример названия: pif.0.clean.topo.toar_LC81120272014074LGN00_B2| [007_Normalization.ipynb](007_Normalization.ipynb#Implementation) 
4b| Аггрегированные псевдоинвариантные пиксели сцены| Берутся растры из 4a для выбранной сцены и | landsat| Пример названия: agg.pif.clean.topo.toar_LC81120272014074LGN00_B2| [007_Normalization.ipynb](007_Normalization.ipynb#Implementation) 

## Векторные

Данные| Описание |Location/Mapset | Название карты|Документ
-----|--|-----------------------|-
Вырубки | Обучающие полигоны по вырубкам за 2015-2016 гг.|FOREST/PERMANENT | train15_16|[000_Initialization.ipynb](000_Initialization.ipynb)
Не-Вырубки | Обучающие полигоны по не-вырубкам за 2015-2016 гг.|FOREST/PERMANENT | nochange15_16|[000_Initialization.ipynb](000_Initialization.ipynb)

